# 2. OpenAI의 챗 API 기초


## 2.3. 입출력 길이 제한과 요금에 영향을 미치는 "토큰"


### 토큰


In [1]:
!pip install tiktoken==0.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.9 MB/s eta 0:00:00


In [2]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))

Chat
GPT


### 토크나이저와 tiktoken 소개


In [19]:
import tiktoken

text = "LLM을 사용해 멋진 것을 만들기는 쉽지만, 프로덕션에서 사용할 수 있는 것을 만들기는 매우 어렵다."

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))

28


## 2.4. Chat Completions API 테스트 환경 준비


### OpenAI API 키 준비


In [4]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 2.5. Chat Completions API 실습


### OpenAI 라이브러리


#### 【주의】알려진 오류에 관해

openai 패키지가 의존하는 httpx의 업데이트로 인해, `openai==1.40.6`을 사용하는 부분에서 `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'`라는 오류가 발생하게 되었습니다.

이 오류는 `!pip install httpx==0.27.2`와 같이, httpx의 특정 버전을 설치함으로써 회피할 수 있습니다.

또한, Google Colab에서 위 오류를 한 번 만난 후 `!pip install httpx==0.27.2`와 같이 패키지를 다시 설치한 경우, 다음 중 하나의 작업을 수행해야 합니다.

- Google Colab의 "런타임"에서 "세션 재시작하기"를 실행합니다
- "런타임 연결 해제 및 삭제"를 실행하여 패키지 설치부터 다시 시작합니다


In [5]:
!pip install openai==1.40.6 httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


### Chat Completions API 호출


In [6]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕하세요! 저는 존이라고 합니다!"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-B9SHSJ0nLoPD5Ip9NkqlkSKg4Si13",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "안녕하세요, 존! 만나서 반갑습니다. 어떻게 도와드릴까요?",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1741593362,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_06737a9306",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 25,
    "total_tokens": 44,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### 대화 이력을 고려한 응답 얻기


In [7]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕하세요! 저는 존이라고 합니다!"},
        {"role": "assistant", "content": "안녕하세요, 존 님! 만나서 반갑습니다. 오늘은 어떤 이야기를 나눌까요?"},
        {"role": "user", "content": "제 이름을 아시나요?"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-B9SHSraEmdiQObBPk3s8bPnx3rEsK",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "네, 존 님이라고 하셨죠! 이름을 알고 있어요. 도움이 필요하시면 언제든지 말씀해 주세요.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1741593362,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_06737a9306",
  "usage": {
    "completion_tokens": 28,
    "prompt_tokens": 63,
    "total_tokens": 91,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### 스트리밍으로 응답 얻기


In [8]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕하세요! 저는 존이라고 합니다!"},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)

안녕하세요, 존님! 어떻게 도와드릴까요?

### JSON 모드


In [9]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '인물 목록을 다음 JSON 형식으로 출력해 주세요.\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content": "옛날 옛적에 할아버지와 할머니가 살고 있었습니다",
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

{"people": ["할아버지", "할머니"]}


### Vision(이미지 입력)


In [10]:
from openai import OpenAI

client = OpenAI()

image_url = "https://raw.githubusercontent.com/ychoi-kr/langchain-book/main/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "이미지를 설명해 주세요."},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)

이미지는 책 표지로 보이며, 제목은 "챗GPT와 랭체인을 활용한 LLM 기반 AI 앱 개발"입니다. 디자인에는 다양한 그래픽 요소가 포함되어 있고, 주로 파란색과 노란색 계열의 색상이 사용된 것 같습니다. 중앙에는 여러 형태의 기하학적 모양과 곤충처럼 보이는 캐릭터가 배치되어 있습니다. 전반적으로 프로그래밍과 AI 개발에 관련된 내용을 다루고 있는 것으로 보입니다.


### (칼럼) Completions API


In [11]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="안녕하세요! 저는 존이라고 합니다!",
)
print(response.to_json(indent=2))

{
  "id": "cmpl-B9SHYJhpFCee52ckCtuZvaTIt92wZ",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n반가워요, 저도 처음 만나서"
    }
  ],
  "created": 1741593368,
  "model": "gpt-3.5-turbo-instruct:20230824-v2",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 17,
    "total_tokens": 33
  }
}


## 2.6. Function calling


### Function calling 샘플 코드


In [20]:
import json


def get_current_weather(location, unit="fahrenheit"):
    if "seoul" in location.lower():
        return json.dumps({"location": "Seoul", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [21]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [22]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "서울 날씨는 어떤가요?"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-B9T2z45KQjB3M7JlIfg8a8F6BQiLy",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_VPAL6aDof51Hck9OOyKG6hKo",
            "function": {
              "arguments": "{\"location\":\"Seoul, KR\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1741596309,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_eb9dce56a8",
  "usage": {
    "completion_tokens": 24,
    "prompt_tokens": 80,
    "total_tokens": 104,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,

In [23]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

In [24]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 사용하고 싶은 함수는 여러 개일 수 있으므로 반복문 사용
for tool_call in response_message.tool_calls:
    # 함수를 실행
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # 함수 실행 결과를 대화 이력으로 messages에 추가
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

{"location": "Seoul", "temperature": "10", "unit": "celsius"}


In [25]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "서울 날씨는 어떤가요?"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_VPAL6aDof51Hck9OOyKG6hKo",
        "function": {
          "arguments": "{\"location\":\"Seoul, KR\",\"unit\":\"celsius\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_VPAL6aDof51Hck9OOyKG6hKo",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Seoul\", \"temperature\": \"10\", \"unit\": \"celsius\"}"
  }
]


In [26]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(second_response.to_json(indent=2))

{
  "id": "chatcmpl-B9T30F1NsCNuPA2bt3s5BUFqk16Gf",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "현재 서울의 기온은 약 10도입니다. 더 구체적인 날씨 정보가 필요하시면 말씀해 주세요.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1741596310,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_eb9dce56a8",
  "usage": {
    "completion_tokens": 28,
    "prompt_tokens": 67,
    "total_tokens": 95,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}
